In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train_data = pd.read_csv('train_data.csv', encoding='gbk') # 读取数据集
 
filter_feature = ['id','label'] # 过滤无用的维度
features = []
for x in train_data.columns: # 取特征
    if x not in filter_feature:
        features.append(x)
 
train_data_x = train_data[features] 
train_data_y = train_data['label']
X_train, X_test, y_train, y_test = train_test_split(train_data_x, train_data_y, random_state=1) # 划分训练集、测试集

In [ ]:
train_data.fillna(0, inplace=True) # 填充 0

In [ ]:
train_data.fillna(train_data.mean(),inplace=True) # 填充均值

In [ ]:
train_data.fillna(train_data.median(),inplace=True) # 填充中位数

In [ ]:
train_data.fillna(train_data.mode(),inplace=True) # 填充众数,该数据缺失太多众数出现为nan的情况
features_mode = {}
for f in features:
    print f,':', list(train_data[f].dropna().mode().values)
    features_mode[f] = list(train_data[f].dropna().mode().values)[0]
train_data.fillna(features_mode,inplace=True)

In [ ]:
for f in features: # 插值法填充
    train_data[f] = train_data[f].interpolate()
    
train_data.dropna(inplace=True)

In [ ]:
from fancyimpute import KNN
 
train_data_x = pd.DataFrame(KNN(k=6).fit_transform(train_data_x), columns=features)

In [ ]:

new_label = 'SNP46'
new_features = []
for f in features:
    if f != new_label:
        new_features.append(f)
        
new_train_x = train_data[train_data[new_label].isnull()==False][new_features]
new_train_x.fillna(new_train_x.mean(), inplace=True) # 其他列填充均值
new_train_y = train_data[train_data[new_label].isnull()==False][new_label]
 
new_predict_x = train_data[train_data[new_label].isnull()==True][new_features]
new_predict_x.fillna(new_predict_x.mean(), inplace=True) # 其他列填充均值
new_predict_y = train_data[train_data[new_label].isnull()==True][new_label]
 
rfr = RandomForestRegressor(random_state=666, n_estimators=10, n_jobs=-1)
rfr.fit(new_train_x, new_train_y)
new_predict_y = rfr.predict(new_predict_x)
 
new_predict_y = pd.DataFrame(new_predict_y, columns=[new_label], index=new_predict_x.index)
new_predict_y = pd.concat([new_predict_x, new_predict_y], axis=1)
new_train_y = pd.concat([new_train_x, new_train_y], axis=1)
new_train_data = pd.concat([new_predict_y,new_train_y]) 
 
train_data_x = new_train_data[features]
train_data_y = train_data['label']

In [ ]:
def countF1(train, predict):
    count = 0 # 统计预测的正确的正样本数
    for i in range(len(train)):
        if predict[i] == 1 and train[i] == 1:
            count += 1
    pre =  count * 1.0 / sum(predict) # 准确率
    recall =  count * 1.0 / sum(train) # 召回率
    return 2 * pre * recall / (pre + recall)